In [2]:
# Import data and libraries
import pandas as pd

data = pd.read_csv('data/root_cause_analysis.csv', index_col='ID')
data.head()

,CPU_LOAD,MEMORY_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003,ROOT_CAUSE
ID,,,,,,,,
1,0,0,0,0,1,0,1,MEMORY
2,0,0,0,0,0,0,1,MEMORY
3,0,1,1,0,0,1,1,MEMORY
4,0,1,0,1,1,0,1,MEMORY
5,1,1,0,1,0,1,0,NETWORK_DELAY


In [4]:
# Check the missing value
data.info()


# Check the descriptive analysis
data.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 1 to 1000
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CPU_LOAD     1000 non-null   int64 
 1   MEMORY_LOAD  1000 non-null   int64 
 2   DELAY        1000 non-null   int64 
 3   ERROR_1000   1000 non-null   int64 
 4   ERROR_1001   1000 non-null   int64 
 5   ERROR_1002   1000 non-null   int64 
 6   ERROR_1003   1000 non-null   int64 
 7   ROOT_CAUSE   1000 non-null   object
dtypes: int64(7), object(1)
memory usage: 70.3+ KB


,CPU_LOAD,MEMORY_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.472000,0.418000,0.394000,0.395000,0.485000,0.432000,0.381000
std,0.499465,0.493477,0.488879,0.489095,0.500025,0.495602,0.485876
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
# Set the Feature and Target
X = data.iloc[:, :7]
y = data.iloc[:, -1]

In [8]:
# Importing classifier libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV


# Setting the random state
SEED = 42

In [9]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=SEED,
                                                    test_size=0.33,
                                                    stratify=y)